In [1]:
import pickle
import glob
import numpy as np
from coffea import processor
from coffea import util
from coffea.lookup_tools.dense_lookup import dense_lookup

In [2]:
# folder with b_tag_efficiency_processor output files
output_histograms = "..."

# load btag efficiency processor outputs 
histograms = []
for file in glob.glob(f"{output_histograms}/*.pkl"):
    with open(file, "rb") as f:
        histograms.append(pickle.load(f))

In [3]:
# accumulate all histograms
acc_histograms = processor.accumulate(histograms)
acc_histograms

Hist(
  Regular(20, 20, 500, name='pt'),
  Regular(4, 0, 2.5, name='abseta', label='Jet abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  Regular(2, 0, 2, name='passWP'),
  storage=Double()) # Sum: 5568759449.0 (5597749609.0 with flow)

In [4]:
# compute efficiencies 
eff = acc_histograms[{"passWP": True}] / acc_histograms[{"passWP": sum}]
eff

Hist(
  Regular(20, 20, 500, name='pt'),
  Regular(4, 0, 2.5, name='abseta', label='Jet abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: 80.93964809907604 (nan with flow)

In [5]:
# create lookup table
efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
efflookup

3 dimensional histogram with axes:
	1: [ 20.  44.  68.  92. 116. 140. 164. 188. 212. 236. 260. 284. 308. 332.
 356. 380. 404. 428. 452. 476. 500.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

Note that hadron flavours (0, 4, 5) turn into (0, 1, 2). 

In [6]:
# Efficiency at 43 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(43, 0.1, np.array([0, 1, 2]))

array([0.02550582, 0.19399992, 0.78824355])

Let's redefine the flavor axis to have the original flavour values

In [7]:
efflookup._axes[-1] = np.array([0., 4., 5., 6.])
efflookup

3 dimensional histogram with axes:
	1: [ 20.  44.  68.  92. 116. 140. 164. 188. 212. 236. 260. 284. 308. 332.
 356. 380. 404. 428. 452. 476. 500.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 4. 5. 6.]

In [8]:
# Efficiency at 43 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(43, 0.1, np.array([0, 4, 5]))

array([0.02550582, 0.19399992, 0.78824355])

In [9]:
# save lookup table
output_path = "..."
util.save(efflookup, f"{output_path}/btag_eff_deepJet_M_2017.coffea")